In [4]:
%load_ext autoreload
%autoreload 2

In [4]:
!pip install scikit-image
!pip install sklearn
!pip install scipy
!pip install hdbscan
!pip install seaborn
!pip install hdbscan

You should consider upgrading via the '/Users/trevorchan/environments/machine-learning/bin/python3 -m pip install --upgrade pip' command.
  Using cached sklearn-0.0.tar.gz (1.1 kB)
  Using cached scikit_learn-0.23.1-cp37-cp37m-macosx_10_9_x86_64.whl (7.2 MB)
  Using cached threadpoolctl-2.1.0-py3-none-any.whl (12 kB)
  Using cached joblib-0.15.1-py3-none-any.whl (298 kB)
    Running setup.py install for sklearn ... done
You should consider upgrading via the '/Users/trevorchan/environments/machine-learning/bin/python3 -m pip install --upgrade pip' command.
You should consider upgrading via the '/Users/trevorchan/environments/machine-learning/bin/python3 -m pip install --upgrade pip' command.
Processing /Users/trevorchan/Library/Caches/pip/wheels/12/bb/54/7401fdc7883b5975dba5514a185d75d7a0986472df4957637b/hdbscan-0.8.26-cp37-cp37m-macosx_10_14_x86_64.whl
  Using cached Cython-0.29.20-cp37-cp37m-macosx_10_9_x86_64.whl (1.9 MB)
You should consider upgrading via the '/Users/trevorchan/envir

In [127]:
import sys
import math
import matplotlib.pyplot as plt
import numpy as np
import glob
import os
from PIL import Image
import torch

from tqdm import tqdm

import seaborn as sns
import pickle

import cv2
from PIL import Image

from skimage.draw import ellipse
from skimage.measure import label, regionprops, find_contours, approximate_polygon
from skimage.transform import rotate
from sklearn.neighbors import KDTree
from scipy.spatial import distance

#from sklearn.cluster import DBSCAN
import hdbscan

In [18]:
#load model output into instances dict: 
instances = {}
with open('instances.data', 'rb') as filehandle:
    instances = pickle.load(filehandle)
    
'''   "pred_boxes" "pred_masks" "scores" "classes"  "image_size" '''

'   "pred_boxes" "pred_masks" "scores" "classes"  "image_size" '

In [146]:
def make_adjacencies(instances, mindist = 50):
    #takes a list of n cell_poly objects
    #takes a minimum distance defined as adjacent
    #returns an adjacency list of length n
    
    centroid_list = []
    adjacency_list = []
    wide_list = []

    centroid_list = [ (((box[0]+box[2])/2).item() , ((box[1]+box[3])/2).item()) for box in instances["pred_boxes"] ]
    #print(len(centroid_list))
    centroid_list = np.array(centroid_list)

    polygon_list = [ np.reshape(mask[0], (int(len(mask[0])/2) , 2)) for mask in instances["pred_masks"] ]

    tree = []
    tree = KDTree(centroid_list, leaf_size=10)

    wide_list = tree.query_radius(centroid_list, mindist * 5)

    for i,small_list in tqdm(enumerate(wide_list)):
        adjacency_list.append (list())
        for index in small_list:
            # if we've already explored this pair then skip (as cdist _should_ be symmetrical)
            if index <= i:
                continue
            d = distance.cdist(polygon_list[i],polygon_list[index],'euclidean')
            if min(d.flatten()) < mindist:
                adjacency_list[i].append (index)
    
    return adjacency_list


In [137]:
centroid_list = []
adjacency_list = []
wide_list = []

centroid_list = [ (((box[0]+box[2])/2).item() , ((box[1]+box[3])/2).item()) for box in instances["pred_boxes"] ]
#print(len(centroid_list))
centroid_list = np.array(centroid_list)

polygon_list = [ np.reshape(mask[0], (int(len(mask[0])/2) , 2)) for mask in instances["pred_masks"] ]

tree = []
tree = KDTree(centroid_list, leaf_size=10)

wide_list = tree.query_radius(centroid_list, 200)

for i,small_list in tqdm(enumerate(wide_list)):
    adjacency_list.append (list())
    for index in small_list:
        # if we've already explored this pair then skip (as cdist _should_ be symmetrical)
        if index <= i:
            continue
        d = distance.cdist(polygon_list[i],polygon_list[index],'euclidean')
        if min(d.flatten()) < 50:
            adjacency_list[i].append (index)


0it [00:00, ?it/s]
3it [00:00, 23.14it/s]
6it [00:00, 23.29it/s]
9it [00:00, 24.44it/s]
13it [00:00, 26.15it/s]
16it [00:00, 25.17it/s]
19it [00:00, 24.67it/s]
22it [00:00, 23.59it/s]
25it [00:01, 21.78it/s]
28it [00:01, 17.96it/s]
30it [00:01, 18.35it/s]
32it [00:01, 17.78it/s]
35it [00:01, 18.81it/s]
37it [00:01, 14.74it/s]
40it [00:01, 16.69it/s]
43it [00:02, 17.92it/s]
46it [00:02, 18.97it/s]
49it [00:02, 17.44it/s]
52it [00:02, 18.68it/s]
55it [00:02, 17.50it/s]
57it [00:02, 17.43it/s]
59it [00:03, 18.09it/s]
62it [00:03, 18.63it/s]
64it [00:03, 16.37it/s]
66it [00:03, 13.43it/s]
68it [00:03, 14.71it/s]
70it [00:03, 12.80it/s]
72it [00:03, 13.61it/s]
75it [00:04, 15.74it/s]
78it [00:04, 17.53it/s]
80it [00:04, 17.94it/s]
82it [00:04, 17.52it/s]
84it [00:04, 16.76it/s]
87it [00:04, 17.22it/s]
89it [00:04, 16.39it/s]
91it [00:04, 15.84it/s]
93it [00:05, 15.69it/s]
96it [00:05, 17.66it/s]
99it [00:05, 17.91it/s]
101it [00:05, 18.43it/s]
103it [00:05, 18.38it/s]
106it [00:05, 18.33it

In [145]:
adjacency_list

[[102, 143, 44, 283, 56, 16],
 [],
 [158, 107, 4, 342, 31],
 [62, 210, 405, 157, 408, 277, 361, 281],
 [158, 107, 342, 31],
 [197, 252, 416, 128, 189, 98, 428, 433, 99, 225],
 [99, 247, 225, 92, 207],
 [413, 363, 339, 15, 74, 73, 377, 303],
 [56],
 [211],
 [211, 297, 334, 67],
 [155, 30, 247, 207, 118, 305, 185, 351, 88],
 [13, 121, 273, 66, 178, 20, 52, 160],
 [168, 121, 273, 66, 52, 341, 160],
 [322, 135, 398, 93, 95, 440, 442, 297],
 [413, 60, 220, 363, 339, 364, 74, 73, 377],
 [102, 143, 44, 283, 56],
 [189, 428, 104, 41],
 [209,
  172,
  299,
  108,
  156,
  238,
  406,
  432,
  368,
  196,
  71,
  443,
  327,
  294,
  197,
  252,
  416,
  353,
  208],
 [129, 25, 50, 373, 380, 163, 402],
 [273, 66, 324, 296, 349, 435],
 [140, 215, 42, 118, 305, 351],
 [68,
  72,
  191,
  223,
  261,
  370,
  365,
  383,
  224,
  64,
  39,
  418,
  429,
  201,
  148,
  319,
  310,
  97,
  258],
 [103, 320, 358, 359, 235, 415],
 [269, 243, 133, 213, 41, 259, 260, 422, 315],
 [236, 431, 129, 154, 50,

In [147]:
!git add updated_analysis.ipynb

In [148]:
!git status

On branch master
Your branch is up to date with 'origin/master'.

Changes to be committed:
  (use "git reset HEAD <file>..." to unstage)

	new file:   updated_analysis.ipynb

Changes not staged for commit:
  (use "git add/rm <file>..." to update what will be committed)
  (use "git checkout -- <file>..." to discard changes in working directory)

	modified:   ../.DS_Store
	modified:   morphological_analysis.ipynb
	deleted:    ../morphological_analysis.ipynb

Untracked files:
  (use "git add <file>..." to include in what will be committed)

	../.ipynb_checkpoints/
	.DS_Store
	.ipynb_checkpoints/
	HighConfluencyTrain3_InProgress.png
	Untitled.ipynb
	instances.data
	untitled.txt



In [149]:
!git add morphological_analysis.ipynb

In [ ]:
!git commit -m 